In [ ]:
import kagglehub
path = kagglehub.dataset_download("ahmedxc4/skin-ds")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'skin-ds' dataset.
Path to dataset files: /kaggle/input/skin-ds


In [ ]:
import os
print(os.listdir(path))


['README.md', 'val', 'test', 'train']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
import os

print("Root path:", path)

folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
print("Folders found:", folders)


Root path: /kaggle/input/skin-ds
Folders found: ['val', 'test', 'train']


In [ ]:
for folder in folders:
    print(f"\nContents of {folder}:")
    print(os.listdir(os.path.join(path, folder)))



Contents of val:
['Benign keratosis-like lesions', 'HFMD', 'Actinic keratoses', 'Monkeypox', 'Melanoma', 'Measles', 'Healthy', 'Squamous cell carcinoma', 'Basal cell carcinoma', 'Chickenpox', 'Melanocytic nevi', 'Vascular lesions', 'Dermatofibroma', 'Cowpox']

Contents of test:
['Benign keratosis-like lesions', 'HFMD', 'Actinic keratoses', 'Monkeypox', 'Melanoma', 'Measles', 'Healthy', 'Squamous cell carcinoma', 'Basal cell carcinoma', 'Chickenpox', 'Melanocytic nevi', 'Vascular lesions', 'Dermatofibroma', 'Cowpox']

Contents of train:
['Benign keratosis-like lesions', 'HFMD', 'Actinic keratoses', 'Monkeypox', 'Melanoma', 'Measles', 'Healthy', 'Squamous cell carcinoma', 'Basal cell carcinoma', 'Chickenpox', 'Melanocytic nevi', 'Vascular lesions', 'Dermatofibroma', 'Cowpox']


In [ ]:
train_full = datasets.ImageFolder(path + "/train", transform=train_transform)
test_data  = datasets.ImageFolder(path + "/test", transform=val_test_transform)



In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(train_full))
val_size = len(train_full) - train_size

train_data, val_data = random_split(train_full, [train_size, val_size])


In [ ]:
train_data = datasets.ImageFolder(path + "/train", transform=train_transform)
val_data   = datasets.ImageFolder(path + "/val", transform=val_test_transform)
test_data  = datasets.ImageFolder(path + "/test", transform=val_test_transform)


In [ ]:
num_classes = len(train_full.classes) if 'train_full' in locals() else len(train_data.classes)
print("Number of classes:", num_classes)


Number of classes: 14


In [ ]:
from torchvision import datasets

train_full = datasets.ImageFolder(
    path + "/train",
    transform=train_transform
)

test_data = datasets.ImageFolder(
    path + "/test",
    transform=val_test_transform
)

num_classes = len(train_full.classes)
print("Classes:", train_full.classes)
print("Number of classes:", num_classes)


Classes: ['Actinic keratoses', 'Basal cell carcinoma', 'Benign keratosis-like lesions', 'Chickenpox', 'Cowpox', 'Dermatofibroma', 'HFMD', 'Healthy', 'Measles', 'Melanocytic nevi', 'Melanoma', 'Monkeypox', 'Squamous cell carcinoma', 'Vascular lesions']
Number of classes: 14


In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(train_full))
val_size = len(train_full) - train_size

train_data, val_data = random_split(train_full, [train_size, val_size])


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
import timm

model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=True,
    num_classes=num_classes
)

model = model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=10
)


In [ ]:
from torch.amp import autocast, GradScaler
scaler = GradScaler("cuda")


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):

    # -------- TRAIN --------
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with autocast("cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (preds == labels).sum().item()

    scheduler.step()

    train_loss /= len(train_loader)
    train_acc = 100 * train_correct / train_total

    # -------- VALIDATION --------
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (preds == labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total

    print(
        f"Epoch [{epoch+1}/10] | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%"
    )


Epoch [1/10] | Train Loss: 1.6964, Train Acc: 51.59% | Val Loss: 1.5737, Val Acc: 56.47%
Epoch [2/10] | Train Loss: 1.3987, Train Acc: 62.96% | Val Loss: 1.3335, Val Acc: 65.58%
Epoch [3/10] | Train Loss: 1.2988, Train Acc: 67.15% | Val Loss: 1.3006, Val Acc: 67.67%
Epoch [4/10] | Train Loss: 1.2156, Train Acc: 70.81% | Val Loss: 1.2157, Val Acc: 71.18%
Epoch [5/10] | Train Loss: 1.1241, Train Acc: 74.86% | Val Loss: 1.1339, Val Acc: 73.55%
Epoch [6/10] | Train Loss: 1.0476, Train Acc: 78.02% | Val Loss: 1.1433, Val Acc: 73.69%
Epoch [7/10] | Train Loss: 0.9709, Train Acc: 81.16% | Val Loss: 1.0642, Val Acc: 77.31%
Epoch [8/10] | Train Loss: 0.8835, Train Acc: 85.22% | Val Loss: 1.0448, Val Acc: 78.16%
Epoch [9/10] | Train Loss: 0.7989, Train Acc: 89.28% | Val Loss: 1.0564, Val Acc: 78.55%
Epoch [10/10] | Train Loss: 0.7227, Train Acc: 92.82% | Val Loss: 1.0800, Val Acc: 78.98%


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 79.12%


In [ ]:
torch.save(model.state_dict(), "vit_skin_ds_final.pth")


In [ ]:
torch.save(model.state_dict(), "vit_skin_ds_final.pth")


In [ ]:
import os

save_dir = "/content/drive/MyDrive/ViT_Skin_Project"
os.makedirs(save_dir, exist_ok=True)


In [ ]:
model_path = save_dir + "/vit_skin_ds_10epochs.pth"
torch.save(model.state_dict(), model_path)

print("Model saved to:", model_path)


Model saved to: /content/drive/MyDrive/ViT_Skin_Project/vit_skin_ds_10epochs.pth


In [ ]:
with open(save_dir + "/results.txt", "w") as f:
    f.write("Vision Transformer Results\n")
    f.write("Train Accuracy: 92.82%\n")
    f.write("Validation Accuracy: 78.98%\n")
    f.write("Test Accuracy: 79.12%\n")

print("Results saved.")


Results saved.


In [ ]:
os.listdir(save_dir)


['results.txt', 'vit_skin_ds_10epochs.pth']

In [ ]:
model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=False,
    num_classes=14
)

model.load_state_dict(
    torch.load("/content/drive/MyDrive/ViT_Skin_Project/vit_skin_ds_10epochs.pth")
)

model = model.to(device)
model.eval()


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False